## <u> Custom Permission</u>

```
    .has_permission(self, request, view)
    .has_object_permission(self, request, view, obj)
```

In [ ]:
#example-> checks incoming ip against block ip
# app/permissions.py
class CustomPermission(viewset):
    def has_permission(self, request, view):
        '''
            Every action calls this method
        '''
        ip_addr = request.META['REMOTE_ADDR']
        blocked = Blocklist.objects.filter(ip_addr=ip_addr).exists()
        return not blocked
    
    def has_object_permission(self, request, view, obj):
         """
            Object-level permission to only allow owners of an object to edit it.
            Assumes the model instance has an `owner` attribute.
        """
        # Read permissions are allowed to any request,
        # so we'll always allow GET, HEAD or OPTIONS requests.
        if request.method in permissions.SAFE_METHODS:
            return True

        # Instance must have an attribute named `owner`.
        return obj.user == request.user


### Note:
- The has_object_permission(...) method is called when you call the .get_object() method of the GenericAPIView.

- Hence, only put, patch, delete, get(only one object), action calls has_object_permissionby default.

- For custom action, or overridden action you must explictly call ```self.get_object()``` method.

In [ ]:
#app/view.py
class UserViewSet(viewsets.ModelViewSet):
    queryset = get_user_model().objects.all()
    serializer_class = UserSerializer

    @action(detail=True, permission_classes=[IsSelf])
    def private(self, request, *args, **kwargs):
        user = self.get_object() ## <-- You should call this or else has_object_permission will not be called
       #alternative
        user = self.get_object_or_404(pk) ## wont call has_object_permission
        data = UserPrivateSerializer(user).data
        return Response(data, status=status.HTTP_200_OK)

---

## <u>ViewSets</u>

DRF has two main systems for handling views:

- APIView: This provides some handler methods, to handle the http verbs: get, post, put, patch, and delete.
- ViewSet: This is an abstraction over APIView, which provides actions as methods:

    - list: read only, returns multiple resources (http verb: get). Returns a list of dicts.
    - retrieve: read only, single resource (http verb: get, but will expect an id). Returns a single dict.
    - create: creates a new resource (http verb: post)
    - update/partial_update: edits a resource (http verbs: put/patch)
    - destroy: removes a resource (http verb: delete)


DRF has two main systems for handling views:

 - APIView: This provides some handler methods, to handle the http verbs: get, post, put, patch, and delete.
- ViewSet: This is an abstraction over APIView, which provides actions as methods:

    - list: read only, returns multiple resources (http verb: get). Returns a list of dicts.
    - create: creates a new resource (http verb: post)
    - retrieve (pk): read only, single resource (http verb: get, but will expect an id). Returns a single dict.
    - update/partial_update(pk): edits a resource (http verbs: put/patch)
    - destroy(pk): removes a resource (http verb: delete)


### <u>ViewSet attributes and Methods</u>
[Attribues and Models details](https://www.cdrf.co/3.6/rest_framework.viewsets/ViewSet.html)

In [2]:

'''
    If you are using ModelViewSet and still want to restrict some methods you can add
'''
http_method_names = ['get', 'post', 'head'] 

'''
    permission_classes/get_permissions(): General permission checks based on the current action, request 
    and targeted object. Object level permissions can only be applied to retrieve, modify and deletion
    actions. Permission checks for list and create will be applied to the entire object type.
    In case of list: subject to restrictions in the queryset.)
    In case of create: subject to restrictions in the serializer.)
    In case of retrieve, modify and delete: subject to restrictions in the serializer.
'''
permission_classes = []

'''
    queryset/get_queryset(): Limits the general visibility of existing objects from the database. The 
    queryset limits which objects will be listed and which objects can be modified or deleted. 
    The get_queryset() method can apply different querysets based on the current action.
'''
queryset = User.objects.all().order_by('username')

'\n    queryset/get_queryset(): Limits the general visibility of existing objects from the database. The \n    queryset limits which objects will be listed and which objects can be modified or deleted. \n    The get_queryset() method can apply different querysets based on the current action.\n'

In [ ]:
'''
    Determine which serializer classes to use for ViewSet
'''
serializer_classes = []

### Different Serializer based on different actions

In [ ]:
serializer_classes = {
        'partial_update': UserPartialUpdateSerializer
        # ... other actions
}
default_serializer_class = UserListSerializer # Your default serializer

def get_serializer_class(self):
    return self.serializer_classes.get(self.action, self.default_serializer_class)

### Note: The better alternaive to http_methods_name is to just import requiered mixin classes from DRF

### Different Permissions based on different Action

In [ ]:
def get_permissions(self):
    if self.action == 'list' or self.action == 'change_role':
        permission_classes = [IsAuthenticated,isScrumMaster]
    else:
        permission_classes = [IsAuthenticated,IsAuthorOrReadOnly,]
    return [permission() for permission in permission_classes]

---

### <u>Unit testing in DRF</u>

[Unit Testing](https://www.aurigait.com/blog/unit-testing-in-django/)

---